# Nuclei Segmentation

Let's begin with nuclei segmentation.
We can either segment the DAPI channel and then filter out the healthy nuclei, segment only the healthy nuclei or go for the ISL1 channel that has the label for the nuclei of interest.

Let's select an image and then crop out a small piece of it to make testing easier.

In [11]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from czifile import CziFile
from napari import Viewer
from scipy.ndimage import distance_transform_edt

In [2]:
DATA_DIR = Path("../data/NS#1_Healhty-DAPI, WGA, ISL1, G3BP.czi")
image_handle = CziFile(DATA_DIR)
img = np.squeeze(image_handle.asarray())[3, :, 1000:1500, 1000:1500]

In [5]:
scale = {
    values_dict["Id"]: values_dict["Value"] * 10**6
    for values_dict in image_handle.metadata(raw=False)["ImageDocument"]["Metadata"][
        "Scaling"
    ]["Items"]["Distance"]
}
spacing = (scale["Z"], scale["Y"], scale["X"])

## Cellpose

Let's begin by trying out cellpose.

In [ ]:
from cellpose import models  # noqa
from cellpose.io import logger_setup  # noqa

logger_setup()

In [ ]:
model = models.Cellpose(model_type="nuclei")

After a couple attempts with several algorithms, I thought there might be issues with all the saturated nuclei.
To circunvent this, I made a mask for those pixels, calculated the distance transform and added this to the original image in order to have some values here.

In [ ]:
saturated_mask = img[3] == 255
distance_saturated = np.asarray(
    [distance_transform_edt(this_plane) for this_plane in saturated_mask]
)

In [ ]:
img = img.astype(float)
img[3] = img[3] + 10 * distance_saturated
img[3] = img[3] / (np.max(img[3]) + 10)

In [ ]:
masks, flows, styles, diams = model.eval(
    [img],
    batch_size=8,
    channel_axis=0,
    diameter=100,
    z_axis=1,
    do_3D=False,
    channels=[3, 0],
    stitch_threshold=0.5,
    normalize=False,
)

In [ ]:
z = 10

plt.imshow(img[3][z])
plt.colorbar()
plt.show()
plt.imshow(masks[0][z])

I still had no success here.

## StarDist

Maybe stardist might work.
Not tested yet.

## Clesperanto Voronoi Otsu

Maybe Clesperanto's Voronoi and Otsu labelling works well here.

In [ ]:
import pyclesperanto_prototype as cle  # noqa

cle.available_device_names()

In [ ]:
input_gpu = cle.push(img[3])
input_gpu

I was unable to make clesperanto work on the laptop.

## Thresholding

Let's go for typical thresholding techniques as nuclei might be separated enough to not need watershed or maybe a simple watershed.

In [18]:
from skimage.feature import peak_local_max  # noqa
from skimage.filters import median, scharr, threshold_otsu  # noqa
from skimage.morphology import ball, binary_erosion, dilation, disk  # noqa
from skimage.segmentation import watershed  # noqa

Let's begin with some median filtering to enhance a bit the edges.
I'm using my own footprint.

And then use Otsu thresolding.

In [55]:
pixel_resolution = 0.5 / scale["X"]

footprint = np.stack(
    [
        np.zeros_like(disk(pixel_resolution)),
        disk(pixel_resolution),
        np.zeros_like(disk(pixel_resolution)),
    ]
)
midpoint = footprint.shape[1] // 2
footprint[0, midpoint, midpoint] = 1
footprint[2, midpoint, midpoint] = 1

filtered_image = median(img, footprint=footprint)
threshold = threshold_otsu(filtered_image)
nuclei = filtered_image > threshold

In [8]:
viewer = Viewer()
viewer.add_image(img, scale=spacing)
viewer.add_labels(nuclei, scale=spacing)

<Labels layer 'nuclei' at 0x7c8d8b380ce0>

Let's apply watershed to get a better separation between nuclei.
We will need the seeds.

In [64]:
footprint = np.stack([np.zeros_like(disk(1)), disk(1), np.zeros_like(disk(1))])

inter = binary_erosion(nuclei, footprint=footprint)

transformed = distance_transform_edt(inter, sampling=spacing)

distance_between_nuclei = 1.7  # in um
pixel_min_distance = np.floor(distance_between_nuclei / scale["X"]).astype(int)

maxima = peak_local_max(transformed, min_distance=pixel_min_distance)

In [65]:
print(pixel_min_distance, len(maxima))

13 77


We need an image for the edges of the nuclei to limit the watershed.

In [58]:
edges = scharr(filtered_image)

In [59]:
viewer = Viewer()
viewer.add_image(transformed, scale=spacing)
viewer.add_image(edges, scale=spacing)
viewer.add_points(
    maxima,
    name="points",
    scale=spacing,
    size=40,
    n_dimensional=True,  # points have 3D "extent"
    face_color="red",
)

<Points layer 'points' at 0x7c8cf3b9cfb0>

In [68]:
markers = np.zeros(img.shape, dtype=np.uint32)
marker_indices = tuple(np.round(maxima).astype(int).T)
markers[marker_indices] = np.arange(len(maxima)) + 1
markers = dilation(markers, ball(5))

segmented = watershed(
    -transformed,
    markers,
    mask=nuclei,
)

In [69]:
viewer = Viewer()
viewer.add_image(img, scale=spacing)
viewer.add_labels(segmented, scale=spacing)

<Labels layer 'segmented' at 0x7c8cb9a0bef0>

It's not the best and should be improved, but it's not that bad.